# Import Data
- Get all data into a single dataframe.
- Each row will represent a __single season's statistics__ from a single player.
- Most players will have multiple rows, as most played multiple seasons.

- Dropping features that won't impact on our predictions. This includes player_id, position (everyone is a QB) and franchise_id.
- Also dropping duplicate columns that appear in both the passing and rushing data.

In [ ]:
# get passing data
pass_df = load_pass_data()

# duplicate players, unique QBs
pass_df.duplicated().sum(), pass_df['player'].nunique()

(0, 288)

- No duplicate players.
- 288 unique NFL quarterbacks.

In [ ]:
# get rushing data
rush_df = load_rush_data()

# duplicate players, unique QBs
# rush_df = rush_df.drop_duplicates()
rush_df.duplicated().sum(), rush_df['player'].nunique()

(0, 277)

- No duplicate players.
- 275 unique NFL quarterbacks.

### Create master df

In [12]:
pass_df.shape, rush_df.shape

((1306, 38), (1254, 28))

In [13]:
# left join to ensure we have players with passing data, as this is more important than rushing data
df = pass_df.merge(rush_df, on=['player', 'year'], how='left')

df.shape, len(set(df.player))

((1306, 64), 288)

- 1306 QB seasons, 64 columns, 288 unique QBs.

# Passing TDs and Yds per Game
- Adding this feature. Total touchdowns/yards are great indicators of performance, but normalizing them (dividing by games played) can avoid punishing players who get injured or miss games for other reasons.

In [14]:
# add column
df['pass_tds_gm'] = df['pass_touchdowns'] / df['player_game_count']

In [15]:
# add column
df['pass_yds_gm'] = df['pass_yards'] / df['player_game_count']

In [16]:
# check
df[['player', 'pass_yds_gm', 'pass_tds_gm', 'year']].head()

,player,pass_yds_gm,pass_tds_gm,year
0,Brett Favre,244.0000,1.1250,2006
1,Jon Kitna,263.0625,1.3125,2006
2,Marc Bulger,268.9375,1.5000,2006
3,Peyton Manning,275.0625,1.9375,2006
4,Drew Brees,276.3125,1.6250,2006


# Experience
- Adding a feature "exp" to represent the player's __years of experience__. For example, a rookie quarterback would have exp=0. 

### QBs who have 2006 data
- Most of these QBs are not rookies.
- Through external research, I manually found the experience of players in the 2006 season. 

In [17]:
# manual input found from research
qbs_2006_exp = [15, 10, 6, 8, 5, 2, 3, 6, 6, 3, 2, 11, 2, 4, 1, 14, 9, 2, 1, 4, 5, 0, 8, 0, 3, 0, 9, 7, 1, 13, 10, 4, 1, 13, 7, 
               7, 3, 12, 13, 3, 0, 7, 1, 3, 6, 5, 11, 0, 4, 3, 8, 5, 5, 8, 2, 13, 4, 3, 1, 5, 0, 1, 0, 19, 12, 7, 6, 3, 4, 0]

# append NaNs to fill out list for all non-2006 players
for _ in range(len(df) - len(qbs_2006_exp)):
    qbs_2006_exp.append(np.nan)

### Add experience col

In [18]:
# add experience col to df
df['exp'] = qbs_2006_exp

# check
df[['player', 'exp', 'year']]

,player,exp,year
0,Brett Favre,15.0,2006
1,Jon Kitna,10.0,2006
2,Marc Bulger,6.0,2006
3,Peyton Manning,8.0,2006
4,Drew Brees,5.0,2006
...,...,...,...
1301,C.J. Beathard,NaN,2023
1302,Sam Darnold,NaN,2023
1303,Mike White,NaN,2023
1304,Aaron Rodgers,NaN,2023


- QBs who played in 2006 season have an "exp" value, non-2006 QBs have NaN value.

In [19]:
# function to fill in experience column
def fill_experience(group):
    # get first experience value for a player
    first_exp = group['exp'].iloc[0]
    
    # if value is null, set to 0 (rookie season)
    if pd.isna(first_exp):
        first_exp = 0
    
    # define range of years to fill each player's experience column
    experience = range(int(first_exp), int(first_exp) + len(group))
    group['exp'] = list(experience)
    return group

# group by index and fill experience values, drop multi-index
df = df.groupby('player').apply(fill_experience).droplevel(0)

In [20]:
# check
df[df['player'] == 'Patrick Mahomes'][['player', 'year', 'exp']]

,player,year,exp
853,Patrick Mahomes,2017,0
875,Patrick Mahomes,2018,1
956,Patrick Mahomes,2019,2
1012,Patrick Mahomes,2020,3
1090,Patrick Mahomes,2021,4
1166,Patrick Mahomes,2022,5
1250,Patrick Mahomes,2023,6


- All players now have a corresponding experience for each of their seasons.

# Past Performance
- Engineer two features that capture a QB's historical performance over their career (all past seasons and their current season).
- I will look at each player's offensive grade over their 3 most recent seasons, and use the mean (capturing peformance) along with the standard deviation (capturing consistency).

In [21]:
# ensure values are sorted properly
df = df.sort_values(by=['player', 'exp']).reset_index(drop=True)
df[['player', 'year', 'exp']].head()

,player,year,exp
0,A.J. Feeley,2006,5
1,A.J. Feeley,2007,6
2,A.J. Feeley,2011,7
3,A.J. McCarron,2015,0
4,A.J. McCarron,2017,1


In [22]:
# use current season's offensive grade + 2 previous seasons
window_size = 3

# calculate rolling mean and std
# 'min_periods=1' ensures we calculate statistics even if there's only one past season. drop multi-index with droplevel()
df['rolling_mean'] = df.groupby('player')['pass_grades_offense'].apply(
    lambda x: x.rolling(window=window_size, min_periods=1).mean()).droplevel(0)
df['rolling_std'] = df.groupby('player')['pass_grades_offense'].apply(
    lambda x: x.rolling(window=window_size, min_periods=1).std()).droplevel(0)

# handle NaN values
df['rolling_mean'].fillna(0, inplace=True)
df['rolling_std'].fillna(0, inplace=True)

In [23]:
# check
df[df['player'] == 'Patrick Mahomes'][['player', 'year', 'pass_grades_offense', 'rolling_mean', 'rolling_std']]

,player,year,pass_grades_offense,rolling_mean,rolling_std
993,Patrick Mahomes,2017,72.6,72.600000,0.000000
994,Patrick Mahomes,2018,93.2,82.900000,14.566400
995,Patrick Mahomes,2019,83.6,83.133333,10.307926
996,Patrick Mahomes,2020,91.4,89.400000,5.102940
997,Patrick Mahomes,2021,77.1,84.033333,7.159842
998,Patrick Mahomes,2022,91.3,86.600000,8.227393
999,Patrick Mahomes,2023,87.1,85.166667,7.294747


- Now we have two more engineered features that look at a QBs recent performance in their career.
- Hopefully, this will give our models insight during the training process.

# Nulls

In [24]:
# look at null counts in each col
df.isna().sum().to_frame().T

,player,team_name,player_game_count,pass_accuracy_percent,pass_aimed_passes,pass_attempts,pass_avg_depth_of_target,pass_avg_time_to_throw,pass_bats,pass_big_time_throws,pass_btt_rate,pass_completion_percent,pass_completions,pass_def_gen_pressures,pass_drop_rate,pass_dropbacks,pass_drops,pass_first_downs,pass_grades_hands_fumble,pass_grades_offense,pass_grades_pass,pass_hit_as_threw,pass_interceptions,pass_passing_snaps,pass_penalties,pass_pressure_to_sack_rate,pass_qb_rating,pass_sack_percent,pass_sacks,pass_scrambles,pass_spikes,pass_thrown_aways,pass_touchdowns,pass_turnover_worthy_plays,pass_twp_rate,pass_yards,pass_ypa,year,rush_attempts,rush_avoided_tackles,rush_breakaway_attempts,rush_breakaway_percent,rush_breakaway_yards,rush_designed_yards,rush_elu_recv_mtf,rush_elu_rush_mtf,rush_elu_yco,rush_elusive_rating,rush_explosive,rush_first_downs,rush_fumbles,rush_gap_attempts,rush_grades_offense_penalty,rush_grades_run,rush_longest,rush_run_plays,rush_scramble_yards,rush_total_touches,rush_touchdowns,rush_yards,rush_yards_after_contact,rush_yco_attempt,rush_ypa,rush_zone_attempts,pass_tds_gm,pass_yds_gm,exp,rolling_mean,rolling_std
0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,17,0,0,0,0,0,0,0,0,0,0,40,0,0,0,0,0,0,0,0,0,0,0,0,97,97,97,109,97,97,97,97,149,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,0,0,0,0,0


- Few nulls in the dataset.
- When looking at a players statistics, null values in these columns indicate 0 attempts for that specific statistic.
- These values can be filled with 0.

In [25]:
# fill nulls with value of 0
df = df.fillna(0)

# Target Variable
- For each season __x__, the target is the grades_offense stat of season __x + 1__.

In [26]:
# add target col
df['target'] = df.groupby('player')['pass_grades_offense'].shift(-1)

# check
df.head(3)[['player', 'pass_grades_offense', 'target']]

,player,pass_grades_offense,target
0,A.J. Feeley,64.3,41.2
1,A.J. Feeley,41.2,50.0
2,A.J. Feeley,50.0,NaN


- Players in their final season will have a NaN value for the target variable (as seen above).
- These rows cannot be used to train model, as we don't have the grades_offense stat of the next year (it doesn't exist).

In [27]:
# drop rows with null target (players in their final season and 2023 season)
df = df.dropna(subset=['target'])

In [28]:
# export data
df.to_csv('./train_data/data.csv')

- Now that we've loaded, combined, and cleaned the data (and added new features), let's look at the distribution and correlations within the data in [this notebook](./feature_analysis.ipynb).

In [30]:
df.shape

(1018, 70)

In [31]:
df

,player,team_name,player_game_count,pass_accuracy_percent,pass_aimed_passes,pass_attempts,pass_avg_depth_of_target,pass_avg_time_to_throw,pass_bats,pass_big_time_throws,pass_btt_rate,pass_completion_percent,pass_completions,pass_def_gen_pressures,pass_drop_rate,pass_dropbacks,pass_drops,pass_first_downs,pass_grades_hands_fumble,pass_grades_offense,pass_grades_pass,pass_hit_as_threw,pass_interceptions,pass_passing_snaps,pass_penalties,pass_pressure_to_sack_rate,pass_qb_rating,pass_sack_percent,pass_sacks,pass_scrambles,pass_spikes,pass_thrown_aways,pass_touchdowns,pass_turnover_worthy_plays,pass_twp_rate,pass_yards,pass_ypa,year,rush_attempts,rush_avoided_tackles,rush_breakaway_attempts,rush_breakaway_percent,rush_breakaway_yards,rush_designed_yards,rush_elu_recv_mtf,rush_elu_rush_mtf,rush_elu_yco,rush_elusive_rating,rush_explosive,rush_first_downs,rush_fumbles,rush_gap_attempts,rush_grades_offense_penalty,rush_grades_run,rush_longest,rush_run_plays,rush_scramble_yards,rush_total_touches,rush_touchdowns,rush_yards,rush_yards_after_contact,rush_yco_attempt,rush_ypa,rush_zone_attempts,pass_tds_gm,pass_yds_gm,exp,rolling_mean,rolling_std,target
0,A.J. Feeley,PHI,2,80.6,36,38,8.4,2.28,1,1,2.4,68.4,26,10,10.3,40,3,15,79.5,64.3,66.7,0,0,44,2,10.0,122.9,2.5,1,1,0,1,3,1,2.3,342,9.0,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,1.0,0.0,0.0,55.7,3.0,17.0,3.0,0.0,0.0,3.0,0.0,0.00,3.0,0.0,1.500000,171.000000,5,64.30,0.000000,41.2
1,A.J. Feeley,PHI,3,68.0,97,103,10.5,2.76,2,4,3.8,57.3,59,27,10.6,111,7,35,67.1,41.2,42.2,0,8,114,0,11.1,61.2,2.7,3,5,0,4,5,9,7.9,681,6.6,2007,2.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,82.9,50.4,7.0,71.0,18.0,2.0,0.0,23.0,0.0,0.00,3.3,0.0,1.666667,227.000000,6,52.75,16.334167,50.0
3,A.J. McCarron,CIN,6,81.1,106,119,8.9,2.76,1,5,4.0,66.4,79,53,8.1,139,7,48,66.5,69.7,72.5,2,2,147,1,22.6,97.1,8.6,12,8,0,10,6,2,1.4,854,7.2,2015,6.0,1.0,0.0,0.0,0.0,-16.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,48.7,44.5,16.0,110.0,37.0,6.0,0.0,21.0,6.0,0.43,1.5,0.0,1.000000,142.333333,0,69.70,0.000000,37.1
4,A.J. McCarron,CIN,2,58.3,12,14,8.1,3.11,0,0,0.0,50.0,7,5,0.0,15,0,3,73.9,37.1,38.0,1,0,16,0,20.0,63.4,6.7,1,0,0,1,0,0,0.0,66,4.7,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,33.000000,1,53.40,23.051681,37.2
5,A.J. McCarron,OAK,1,100.0,1,3,6.0,3.60,0,0,0.0,33.3,1,3,0.0,4,0,0,18.7,37.2,54.2,0,0,4,0,33.3,42.4,25.0,1,0,0,2,0,0,0.0,8,2.7,2018,4.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,72.9,40.9,4.0,8.0,0.0,4.0,0.0,2.0,3.0,0.75,0.5,1.0,0.000000,8.000000,2,48.00,18.792818,58.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,Vince Young,TEN,9,69.4,147,156,12.0,0.00,3,15,9.1,59.6,93,59,8.8,182,9,63,26.2,67.2,69.9,2,3,192,2,22.0,98.6,7.1,13,13,0,4,10,11,5.7,1255,8.0,2010,12.0,2.0,1.0,16.0,20.0,14.0,0.0,1.0,10.0,6.9,5.0,6.0,6.0,0.0,34.1,48.4,20.0,188.0,111.0,12.0,0.0,125.0,34.0,1.36,5.0,0.0,1.111111,139.444444,4,58.80,19.507178,51.0
1297,Vinny Testaverde,NE,1,100.0,3,3,4.7,1.80,0,0,0.0,66.7,2,0,33.3,3,1,2,73.3,61.2,70.2,0,0,3,0,0.0,137.5,0.0,0,0,0,0,1,0,0.0,29,9.7,2006,8.0,0.0,0.0,0.0,0.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.6,54.6,-1.0,8.0,0.0,8.0,0.0,-8.0,0.0,0.00,-1.0,0.0,1.000000,29.000000,19,61.20,0.000000,46.9
1301,Zach Mettenberger,TEN,7,68.5,168,179,9.1,2.63,6,8,4.3,59.8,107,75,7.0,198,8,58,90.1,56.7,56.5,2,7,210,1,24.0,83.4,9.1,18,1,0,3,8,8,3.8,1412,7.9,2014,4.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,53.9,61.2,8.0,97.0,8.0,4.0,0.0,4.0,16.0,3.20,0.8,0.0,1.142857,201.714286,0,56.70,0.000000,38.4
1303,Zach Wilson,NYJ,13,69.9,349,383,8.0,3.05,9,10,2.5,55.6,213,171,12.7,444,31,118,69.3,59.3,54.8,3,11,469,1,26.3,69.7,10.1,45,16,1,21,9,18,3.8,2334,6.1,2021,12.0,6.0,0.0,0.0,0.0,6.0,0.0,1.